## Task I
### Question: 
Build and populate necessary tables
#### Core Code with expected output:

In [1]:
import pyspark
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
import pandas as pd

spark = SparkSession.builder \
    .appName("fifa") \
    .config("spark.driver.extraClassPath", "/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/pyspark/jars/postgresql-42.7.4.jar") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 22:57:19 INFO SparkEnv: Registering MapOutputTracker
24/11/11 22:57:19 INFO SparkEnv: Registering BlockManagerMaster
24/11/11 22:57:19 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/11/11 22:57:20 INFO SparkEnv: Registering OutputCommitCoordinator


In [4]:
# Read one of data to get the headers
data_sample = pd.read_csv("gs://fifa-dataset-project-option-i/data/players_15.csv", nrows=2)
column_names = list(data_sample.columns)

In [49]:
# def infer_sql_type(dtype):
#     """Map pandas data type to PostgreSQL"""
#     if pd.api.types.is_integer_dtype(dtype):
#         return "INT"
#     elif pd.api.types.is_float_dtype(dtype):
#         return "FLOAT"
#     elif pd.api.types.is_bool_dtype(dtype):
#         return "BOOLEAN"
#     else:
#         return "VARCHAR"

# # Infer data type for each column in Postgres
# column_types = {col: infer_sql_type(dtype) for col, dtype in zip(data_sample.columns, data_sample.dtypes)}

# # Create a new column of year
# column_types['year'] = "INT"

# # Create a new column of gender
# column_types['gender'] = "VARCHAR"

# # Print the crrent result
# for col, col_type in column_types.items():
#     print(f"{col}: {col_type}")

In [48]:
# # Create a new table names "fifa"
# table = "dataset" 
# columns = ",\n    ".join([f'"{col}" {dtype}' for col, dtype in column_types.items()])

# create_table = f"""
# CREATE SCHEMA fifa;
# CREATE TABLE fifa.{table} (
#     id SERIAL PRIMARY KEY,
#     {columns}
# );
# """

# print(create_table)

In [6]:
# Read male data
total_data = spark.read.csv('gs://fifa-dataset-project-option-i/data/players_15.csv',header=True, inferSchema=True).toDF(*column_names)
total_data = total_data.withColumn("year", lit(2015))
total_data = total_data.withColumn("gender", lit("male"))

for i in range(16,23):
    data = spark.read.csv(f'gs://fifa-dataset-project-option-i/data/players_{i}.csv',header=True, inferSchema=True)
    data = data.toDF(*column_names)
    data = data.withColumn("year", lit(2000+i))
    data = data.withColumn("gender", lit("male"))
    total_data = total_data.union(data)
print("total male players: ",total_data.count())

# Read female data
# for i in range(16,23):
#     data = spark.read.csv(f'/Users/jiayi/Desktop/System-ToolChain-AI/Course Project/archive/female_players_{i}.csv',header=True, inferSchema=True)
#     data = data.toDF(*column_names)
#     data = data.withColumn("year", lit(2000+i))
#     data = data.withColumn("gender", lit("female"))
#     total_data = total_data.union(data)
    
# print("total female players: ",total_data.count())   

# # Ensure every record can be uniquely identified in the database table
# print("total unique records: ",total_data.distinct().count())

total male players:  142079


In [47]:
# # Ingest data to PostgresDB
# postgres_url = "p552350988775-08gmnm@gcp-sa-cloud-sql.iam.gserviceaccount.com"
# postgres_properties = {
#     "user": "forward-byte-434113-d3:us-central1:project-system-toolschain", 
#     "password": "postgres",
#     "driver": "org.postgresql.Driver"
# }

# # Write data to DB
# total_data.write \
#     .jdbc(url=postgres_url, table="fifa", mode="overwrite", properties=postgres_properties)

# # Read data from DB
# data_postgres = spark.read \
#     .jdbc(url=postgres_url, table="fifa", properties=postgres_properties)

# data_postgres.printSchema()
# data_postgres.show(1, vertical=True)

In [46]:
# data_postgres.tail(5)

## Task II

### Question 1: 
In Year X, what were the Y clubs that had the highest number of players with
contracts ending in year Z (or after)?
- X is a year between (2015 and 2022, inclusively).
- Y is a positive integer.
- Z is a year that can hold the value of 2023 or a year after it.

#### Core Code with expected output:

***Experiment***

In [9]:
data_postgres = total_data

In [50]:
# Explore "club_contract_valid_until" column to see possible values of Z
# data_postgres.select("club_contract_valid_until").distinct().show()

*Result*

Possible values for Z: 2031, 2026, 2028, 2025, 2023, 2024, 2027

***Query Function***

In [51]:
from pyspark.sql.functions import col
def query_no_clubs(X, Y, Z):
    # Filter the data as needed and group by club_name
    club_counts = data_postgres.filter(f"club_contract_valid_until == {Z}") \
                               .filter(f"year == {X}") \
                               .filter("gender == 'male'") \
                               .groupBy("club_name") \
                               .count() \
                               .sort("count", ascending=False) \
                               .limit(Y)
    return club_counts

***Function Testing***

In [52]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("year", IntegerType(), True),
    StructField("contract_ending", IntegerType(), True),
    StructField("no_of_clubs", IntegerType(), True)
])

# Create an empty DataFrame with the defined schema
resultDF = spark.createDataFrame([], schema=schema)


# data =[]
# X = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
# Z = [2031, 2026, 2028, 2025, 2023, 2024, 2027]

# Specify y: #clubs that had the highest number of players 
# For example
no_of_clubs = query_no_clubs(2022,5,2023)
print(f"In year {2022} and club contract valid until {2023}:")
no_of_clubs.show()

In year 2022 and club contract valid until 2023:


+--------------------+-----+
|           club_name|count|
+--------------------+-----+
|En Avant de Guingamp|   19|
|       Lechia Gdańsk|   17|
| Club Atlético Lanús|   17|
|            Barnsley|   16|
|        Kasimpaşa SK|   16|
+--------------------+-----+



### Question 2: 
In sports, maturity and energy of teams depend on the average age of team players (among other factors). Therefore, it’s important to have a function that can find clubs with such features.
- List the X clubs with the highest (or lowest) average player age for a given year Y.
    - X represents a positive integer, but you should handle a scenario if X is not positive value.
    - Y represents a year between 2015 and 2022 inclusively.
    - Provide the user with the ability to choose if they want the highest average age or the lowest average age.
    - Make sure to handle this scenario as well: if the user requests 5 clubs with highest averages but there are 3 clubs that share the same count at rank number 5, please include all of them in your output

#### Core Code with expected output:

***Query Function***

In [53]:
def top_avg_age_clubs(X, Y, filter):
    avg_by_group = data_postgres.filter(f"year == {Y}") \
                            .filter("gender == 'male'") \
                            .groupBy("club_name") \
                            .agg(avg("age").alias("average_age"))
    
    # Find the distinct avg age
    avg_by_group.select("average_age").distinct().orderBy("average_age")
    
    # Filter from the highest avg age
    if filter == "highest":
        top = avg_by_group.select("average_age").distinct().sort("average_age", ascending=False).limit(X).collect()
        top_boundary = top[X-1][0]
        return avg_by_group.filter(f"average_age >= {top_boundary}").sort("average_age", ascending=False).show()
    
    # Filter from the lowest avg age
    else:
        bottom = avg_by_group.select("average_age").distinct().sort("average_age", ascending=True).limit(X).collect()
        bottom_boundary = bottom[X-1][0]
        return avg_by_group.filter(f"average_age <= {bottom_boundary}").sort("average_age", ascending=True).show()

***Function Testing***

In [54]:
top_avg_age_clubs(5, 2015, "highest")

+--------------------+------------------+
|           club_name|       average_age|
+--------------------+------------------+
|           Cruz Azul|28.071428571428573|
|        Arsenal Tula|             28.04|
|Podbeskidzie Biel...|27.962962962962962|
|       Fenerbahçe SK|             27.88|
|Leones Negros de ...| 27.79310344827586|
+--------------------+------------------+



### Question 3: 
What is the most popular nationality in the dataset for each year? (i.e. display the
most frequent nation for 2015, 2016, etc.).

#### Core Code with expected output:

***Query Function***

In [60]:
def display_popular_nation():
    # Find the minimum and maximum year in the db
    max_year = data_postgres.agg(max("year")).collect()[0][0]
    min_year = data_postgres.agg(min("year")).collect()[0][0]

    # Group nations for each years
    for i in range(min_year,max_year + 1):
        no_nations = data_postgres.filter(f"year == {i}") \
                            .filter("gender == 'male'") \
                            .groupBy("nationality_name") \
                            .count()
        
        max_count = no_nations.agg(max("count")).collect()[0][0]
        print(f"The most popularity nationality & its total players in year {i}:")
        no_of_nations = no_nations.filter(col("count") == max_count).show()
#     return no_nations

#### Function Testing:

In [61]:
display_popular_nation()

The most popularity nationality & its total players in year 2015:


+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1627|
+----------------+-----+



The most popularity nationality & its total players in year 2016:


+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1519|
+----------------+-----+



The most popularity nationality & its total players in year 2017:


...

+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1670|
+----------------+-----+



The most popularity nationality & its total players in year 2021:


+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1685|
+----------------+-----+



The most popularity nationality & its total players in year 2022:


+----------------+-----+
|nationality_name|count|
+----------------+-----+
|         England| 1719|
+----------------+-----+



## Task III

### Data Collection

#### Data Ingestion from the source

In [45]:
# Ingest male player data
male_data = data_postgres.filter("gender == 'male'")
male_data.show(1, vertical=True)

-RECORD 0-------------------------------------------
 sofifa_id                   | 158023               
 player_url                  | https://sofifa.co... 
 short_name                  | L. Messi             
 long_name                   | Lionel Andrés Mes... 
 player_positions            | CF                   
 overall                     | 93                   
 potential                   | 95                   
 value_eur                   | 1.005E8              
 wage_eur                    | 550000.0             
 age                         | 27                   
 dob                         | 1987-06-24           
 height_cm                   | 169                  
 weight_kg                   | 67                   
 club_team_id                | 241.0                
 club_name                   | FC Barcelona         
 league_name                 | Spain Primera Div... 
 league_level                | 1                    
 club_position               | CF             

#### Data Cleaning & Data Preprocessing

***Note:*** prediction target is `value_eur`

**Delete Some irrvelent columns**

In [13]:
from pyspark.sql.functions import col, substring, split
from pyspark.sql.types import IntegerType, FloatType, DoubleType

In [14]:
columns_to_drop = [
    'sofifa_id', 'player_url', 'short_name', 'long_name', 'club_position','club_name', 'dob', 
    'club_team_id', 'league_name', 'club_jersey_number', 'club_loaned_from',
    'nation_team_id', 'nation_jersey_number', 'nationality_id', 'nationality_name', 'nation_position',
    'nation_jersey_name', 'body_type', 'real_face', 'release_clause_eur',
    'player_tags', 'player_traits', 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 
    'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'lwb', 'ldm', 
    'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk', 'player_face_url', 
    'cub_logo_url', 'club_flag_url', 'nation_logo_url', 'nation_flag_url', 'club_logo_url','year','gender', 'player_positions'
]

male_data_filtered = male_data.drop(*columns_to_drop)

# Remaining columns
print("The number of remaining columns: ", len(male_data_filtered.columns))
male_data_filtered.show(1, vertical=True)

The number of remaining columns:  56


-RECORD 0---------------------------------
 overall                     | 93         
 potential                   | 95         
 value_eur                   | 1.005E8    
 wage_eur                    | 550000.0   
 age                         | 27         
 height_cm                   | 169        
 weight_kg                   | 67         
 league_level                | 1          
 club_joined                 | 2004-07-01 
 club_contract_valid_until   | 2018       
 preferred_foot              | Left       
 weak_foot                   | 3          
 skill_moves                 | 4          
 international_reputation    | 5          
 work_rate                   | Medium/Low 
 pace                        | 93         
 shooting                    | 89         
 passing                     | 86         
 dribbling                   | 96         
 defending                   | 27         
 physic                      | 63         
 attacking_crossing          | 84         
 attacking_

**Columns Adjustment**

- contract duration data: `club_contract_valid_until` - `club_joined` = `contract_duration`
- split data into 2 columns: `work_rate` into `attacking` and `defensive` 

In [15]:
# Extract the first 4 digits (year) from 'club_joined' and 'club_contract_valid_until'
df = male_data_filtered \
    .withColumn("joined_year", substring(col("club_joined"), 1, 4).cast("int")) \
    .withColumn("contract_until_year", substring(col("club_contract_valid_until"), 1, 4).cast("int"))

# Calculate the difference in years and create a new column 'contract_duration'
df = df.withColumn("contract_duration", col("contract_until_year") - col("joined_year"))

# Display the results
df = df.drop('club_joined', 'club_contract_valid_until', 'joined_year','contract_until_year')


In [16]:
# Split the 'work_rate' column into 'attacking' and 'defensive' columns
df = df.withColumn("attacking", split(col("work_rate"), "/").getItem(0)) \
       .withColumn("defensive", split(col("work_rate"), "/").getItem(1))

df = df.drop("work_rate")

In [40]:
# Check the updated data
# print("The number of remaining columns: ", len(df.columns))
# df.show(1, vertical=True)

**Data Missing**

In [17]:
# Final columns to be features
nominal_columns = [
    'preferred_foot', 'attacking', 'defensive'
]

numerical_columns = [
    'international_reputation','overall', 'potential', 'wage_eur', 'age', 'height_cm', 'weight_kg',
    'league_level', 'weak_foot', 'skill_moves',
    'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing',
    'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing',
    'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy',
    'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed',
    'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power',
    'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression',
    'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties',
    'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle',
    'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling',
    'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed','contract_duration'
]

target_column = ['value_eur']

In [15]:
# Function to check missing values
# def CheckNullValues():
#     # List of columns with NULL values
#     columns_with_nulls = []

#     # Check nominal columns for NULL values
#     for column in nominal_columns:
#         if df.filter(col(column).isNull()).count() > 0:
#             columns_with_nulls.append(column)

#     # Check numerical columns for NULL values
#     for column in numerical_columns:
#         if df.filter(col(column).isNull()).count() > 0:
#             columns_with_nulls.append(column)

#     # Display the columns with NULL values
#     print(f"{len(columns_with_nulls)} columns with NULL values:", columns_with_nulls)

In [41]:
# Null values BEFORE handling
# print("BEFORE handling data missing ---")
# CheckNullValues()

- drop the row that has **Null value** in `value_eur`

In [18]:
df = df.dropna(subset=["value_eur"])

- fill the **Null value** with medium value for `wage_eur, league_level, pace, shooting, passing, dribbling, defending, physic, mentality_composure`

In [19]:
# Check for missing or non-numeric columns and attempt to cast them
columns_to_check = ['wage_eur', 'league_level', 'mentality_composure']
for column in columns_to_check:
    if column in df.columns:
        # Attempt to cast to FloatType
        df = df.withColumn(column, col(column).cast(FloatType()))
        # Check if casting was successful
        if df.schema[column].dataType not in (IntegerType(), FloatType(), DoubleType()):
            print(f"Warning: Could not cast {column} to numeric type.")
    else:
        print(f"Warning: {column} does not exist in the DataFrame.")

# Recalculate the median values after casting
median_values = {}
for column in ['wage_eur', 'league_level', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'mentality_composure']:
    if column in df.columns and df.schema[column].dataType in (IntegerType(), FloatType(), DoubleType()):
        median_value = df.filter(col(column).isNotNull()).approxQuantile(column, [0.5], 0.0)[0]
        median_values[column] = median_value
    else:
        print(f"Warning: {column} either does not exist or is not a numeric column.")
print(median_values)

# Fill NULLs with median values
df = df.fillna(median_values)

{'wage_eur': 4000.0, 'league_level': 1.0, 'pace': 69.0, 'shooting': 54.0, 'passing': 58.0, 'dribbling': 63.0, 'defending': 55.0, 'physic': 66.0, 'mentality_composure': 59.0}


- fill the **Null value** with 0 (zero) for `goalkeeping_speed` and `contract_duration`

In [20]:
# Fill NULL values with 0 
df = df.fillna({'goalkeeping_speed': 0, 'contract_duration': 0})

In [42]:
# Null values AFTER handling
# print("AFTER handling data missing ---")
# CheckNullValues()

In [43]:
# Check the updated data
# print("The number of remaining columns: ", len(df.columns))
# df.show(1, vertical=True)

**Convert dataset to vectors**

Create a pipline to analyze columns(features) including outliner, boxplot to see feature statistics and normalize dataset to contribute feature vectors 

In [21]:
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
from functools import reduce

In [22]:
class OutcomeCreater(Transformer):  # this defines a transformer that creates the outcome column
    def __init__(self):
        super().__init__()

    def _transform(self, dataset: DataFrame) -> DataFrame:
        # using column 'value_eur' as 'outcome'
        output_df = dataset.withColumn('outcome', col('value_eur').cast(DoubleType()))

                                       
        return output_df

class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in numerical_columns:
            output_df = output_df.withColumn(col_name,col(col_name).cast(DoubleType()))
        return output_df

class DelectOutliers(Transformer):
    def __init__(self):
        super().__init__()

    def column_add(self, a, b):
        return a.__add__(b)
    
    def _transform(self, dataset: DataFrame) -> DataFrame:
        # Identifying the numerical columns in a spark dataframe
        numeric_columns = [column[0] for column in dataset.dtypes if column[1] in ['int', 'double']]

        # Using the `for` loop to create new columns by identifying the outliers for each feature
        for column in numeric_columns:
            # Calculate the quartiles and IQR for the current numerical column
            Q1 = dataset.approxQuantile(column, [0.25], relativeError=0)[0]
            Q3 = dataset.approxQuantile(column, [0.75], relativeError=0)[0]
            IQR = Q3 - Q1

            # Calculate the outlier bounds
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            
            # Create an outlier indicator column for the current column
            isOutlierCol = f'is_outlier_{column}'
            dataset = dataset.withColumn(isOutlierCol, when((col(column) > upper_bound) | (col(column) < lower_bound), 1).otherwise(0))

        # Selecting the specific columns that indicate outliers
        selected_columns = [column for column in dataset.columns if column.startswith("is_outlier")]

        # Adding all the outlier columns into a new column "total_outliers" to see the total number of outliers
        if selected_columns:
            dataset = dataset.withColumn('total_outliers', reduce(self.column_add, [dataset[col] for col in selected_columns]))

            # Dropping the extra columns created above, just to create a clean dataframe without extra columns
            dataset = dataset.drop(*selected_columns)
        
        # Filtering out rows where 'total_outliers' > 0 (indicating that row is an outlier)
        dataset = dataset.filter(col('total_outliers') == 0).drop('total_outliers')

        return dataset

class StatAnalysis(Transformer):
    def __init__(self, numerical_columns=None):
        self.numerical_cols = numerical_columns
        self.analyzed = False

    def _transform(self, dataset: DataFrame) -> DataFrame:
        if not self.analyzed:
            self.analyze(dataset)
            self.analyzed = True 
        return dataset

    def analyze(self, dataset: DataFrame) -> None:
        print("The statistics for numerical_cols:")
        numerical_data = dataset.select(self.numerical_cols)
        numerical_data.summary().show(truncate=False, vertical=True)

        print("The Boxplot for numerical_cols:")
        converted_data = numerical_data.toPandas()
        plt.boxplot(converted_data)
        
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df

def get_preprocess_pipeline():
    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()
    
    # Stage where finding outliers and delete
    stage_delete_outliers = DelectOutliers()

    # Stage where calculate statistics for numerical_cols and plot boxplot
#     stage_boxplot = StatAnalysis(numerical_columns=numerical_columns)

    # Stage where nominal columns are transformed to index columns using StringIndexer
    nominal_id_cols = [x+"_index" for x in nominal_columns]
    nominal_onehot_cols = [x+"_encoded" for x in nominal_columns]
    stage_nominal_indexer = StringIndexer(inputCols = nominal_columns, outputCols = nominal_id_cols)

    # Stage where the index columns are further transformed using OneHotEncoder
    stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = numerical_columns+nominal_onehot_cols
    stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features")

    # Stage where we scale the columns
    stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')

    # Stage for creating the outcome column
    stage_outcome = OutcomeCreater()
    value_eur_assembler = VectorAssembler(inputCols=['outcome'], outputCol='outcome_vector')
    stage_scaler2 = StandardScaler(inputCol='outcome_vector', outputCol='outcome_scaled')

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = nominal_columns+nominal_id_cols+
        nominal_onehot_cols+ numerical_columns + ['vectorized_features'])
    
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[
        stage_typecaster, 
        stage_delete_outliers,
#         stage_boxplot,
        stage_nominal_indexer,
        stage_nominal_onehot_encoder,
        stage_vector_assembler,
        stage_scaler,
        stage_outcome,
        value_eur_assembler,
        stage_scaler2,
        stage_column_dropper])
    
    return pipeline 

In [23]:
# Get the Pipeline, and need to fit pipline to data
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(df)

# Print the DataFrame after data scaling step within the pipeline by pandas table
data_df = preprocess_pipeline_model.transform(df)

24/11/11 23:30:32 WARN DAGScheduler: Broadcasting large task binary with size 1037.6 KiB
24/11/11 23:31:03 WARN DAGScheduler: Broadcasting large task binary with size 1230.0 KiB
24/11/11 23:31:26 WARN DAGScheduler: Broadcasting large task binary with size 1042.2 KiB
24/11/11 23:31:31 WARN DAGScheduler: Broadcasting large task binary with size 1030.5 KiB


In [24]:
# Display plpline's result including "feature vectors" and "outcome"
data_df.select("features", "outcome_scaled").distinct().toPandas()

from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
def vector_to_value(vector):
    return float(vector[0])

vector_to_value_udf = udf(vector_to_value, DoubleType())
data_df = data_df.withColumn("outcome", vector_to_value_udf("outcome_scaled"))
# data_df.select("features", "outcome").show()

24/11/11 23:47:31 WARN DAGScheduler: Broadcasting large task binary with size 1277.2 KiB
24/11/11 23:48:08 WARN DAGScheduler: Broadcasting large task binary with size 1063.2 KiB


### Modeling 

For each classifier/regressor, identify a few tunable parameters for your model and tune the parameters (using proper metric(s)). Then, run the best model (after tuning) on the test data set and record the test accuracy

In [25]:
# Split data into train and test sets
train_df, test_df = data_df.randomSplit([0.7, 0.3], seed=42)

print(f"Training dataset count: {train_df.count()}")
print(f"Testing dataset count: {test_df.count()}")

num_features = len(train_df.select("features").first()["features"])
print(f"Total number of features: {num_features}")

24/11/11 23:48:22 WARN DAGScheduler: Broadcasting large task binary with size 1281.9 KiB


Training dataset count: 37648


24/11/11 23:48:56 WARN DAGScheduler: Broadcasting large task binary with size 1281.9 KiB


Testing dataset count: 15918


24/11/11 23:49:16 WARN DAGScheduler: Broadcasting large task binary with size 1281.3 KiB


Total number of features: 57


#### SparkML

**Linear Regression**

In [26]:
train_df.cache()

DataFrame[value_eur: double, features: vector, outcome: double, outcome_vector: vector, outcome_scaled: vector]

In [27]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Define LinearRegression algorithm
lr = LinearRegression(featuresCol="features", labelCol="outcome")

# Create ParamGrid for Cross Validation
lr_paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.0, 100.0])
             .addGrid(lr.maxIter, [5, 10, 15])
             .build())

# Evaluatation
evaluator = RegressionEvaluator(labelCol="outcome", predictionCol="prediction", metricName="mse")

# Create cross-validation tuning
lr_cv = CrossValidator(estimator=lr, estimatorParamMaps=lr_paramGrid, 
                    evaluator=evaluator, numFolds=5)

# Fit the cross-validation with train dataset
lr_cv_model = lr_cv.fit(train_df)

# Make predictions
predictions = lr_cv_model.transform(test_df)
MSE = evaluator.evaluate(predictions)
print("Linear Regression Model: Mean Squared Error for Test dataset= " + str(MSE))

24/11/11 23:50:40 WARN DAGScheduler: Broadcasting large task binary with size 1288.2 KiB
24/11/11 23:51:21 WARN DAGScheduler: Broadcasting large task binary with size 1313.4 KiB
24/11/11 23:51:24 WARN Instrumentation: [69c812f2] regParam is zero, which might cause numerical instability and overfitting.
24/11/11 23:51:24 WARN DAGScheduler: Broadcasting large task binary with size 1344.4 KiB
24/11/11 23:51:27 WARN DAGScheduler: Broadcasting large task binary with size 1345.5 KiB
24/11/11 23:51:28 WARN Instrumentation: [69c812f2] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
24/11/11 23:51:32 WARN DAGScheduler: Broadcasting large task binary with size 1337.6 KiB
24/11/11 23:51:34 WARN DAGScheduler: Broadcasting large task binary with size 1338.7 KiB
24/11/11 23:51:35 WARN DAGScheduler: Broadcasting large task binary with size 1313.3 KiB
24/11/11 23:51:36 WARN DAGScheduler: Broadcasting large task binary with size 1340.3 KiB
24/11/11 23:51:37 

24/11/11 23:52:53 WARN DAGScheduler: Broadcasting large task binary with size 1337.6 KiB
24/11/11 23:52:54 WARN DAGScheduler: Broadcasting large task binary with size 1338.7 KiB
24/11/11 23:52:56 WARN DAGScheduler: Broadcasting large task binary with size 1340.7 KiB
24/11/11 23:52:57 WARN DAGScheduler: Broadcasting large task binary with size 1341.8 KiB
24/11/11 23:52:58 WARN DAGScheduler: Broadcasting large task binary with size 1313.4 KiB
24/11/11 23:53:01 WARN Instrumentation: [c4f965ba] regParam is zero, which might cause numerical instability and overfitting.
24/11/11 23:53:01 WARN DAGScheduler: Broadcasting large task binary with size 1344.4 KiB
24/11/11 23:53:02 WARN DAGScheduler: Broadcasting large task binary with size 1345.5 KiB
24/11/11 23:53:02 WARN Instrumentation: [c4f965ba] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
24/11/11 23:53:03 WARN DAGScheduler: Broadcasting large task binary with size 1337.6 KiB
24/11/11 23:53:03 

24/11/11 23:53:57 WARN DAGScheduler: Broadcasting large task binary with size 1344.4 KiB
24/11/11 23:53:58 WARN DAGScheduler: Broadcasting large task binary with size 1345.5 KiB
24/11/11 23:53:59 WARN DAGScheduler: Broadcasting large task binary with size 1337.6 KiB
24/11/11 23:54:00 WARN DAGScheduler: Broadcasting large task binary with size 1338.7 KiB
24/11/11 23:54:00 WARN DAGScheduler: Broadcasting large task binary with size 1340.7 KiB
24/11/11 23:54:01 WARN DAGScheduler: Broadcasting large task binary with size 1341.8 KiB
24/11/11 23:54:02 WARN DAGScheduler: Broadcasting large task binary with size 1313.4 KiB
24/11/11 23:54:04 WARN Instrumentation: [1d8ec1c3] regParam is zero, which might cause numerical instability and overfitting.
24/11/11 23:54:04 WARN DAGScheduler: Broadcasting large task binary with size 1344.4 KiB
24/11/11 23:54:05 WARN DAGScheduler: Broadcasting large task binary with size 1345.5 KiB
24/11/11 23:54:05 WARN Instrumentation: [1d8ec1c3] Cholesky solver failed

Linear Regression Model: Mean Squared Error for Test dataset= 0.3244040747133423


**Random Forest Regression**

In [28]:
from pyspark.ml.regression import RandomForestRegressor

# Define Random Forest Regression algorithm
rf = RandomForestRegressor(featuresCol="features", labelCol="outcome")

# Create ParamGrid for Cross Validation
rf_paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [5, 7, 9],)
             .addGrid(rf.numTrees, [2, 5, 10])       
             .build())

# Evaluatation
evaluator = RegressionEvaluator(labelCol="outcome", predictionCol="prediction", metricName="mse")

# Create cross-validation tuning
rf_cv = CrossValidator(estimator=rf, estimatorParamMaps=rf_paramGrid, 
                    evaluator=evaluator, numFolds=5)

# Fit the cross-validation with train dataset
rf_cv_model = rf_cv.fit(train_df)

# Make predictions
predictions = rf_cv_model.transform(test_df)
MSE = evaluator.evaluate(predictions)
print("Random Forest Model: Mean Squared Error for Test dataset= " + str(MSE))

24/11/12 00:02:58 WARN DAGScheduler: Broadcasting large task binary with size 1313.4 KiB
24/11/12 00:02:59 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
24/11/12 00:03:00 WARN DAGScheduler: Broadcasting large task binary with size 1342.0 KiB
24/11/12 00:03:01 WARN DAGScheduler: Broadcasting large task binary with size 1347.0 KiB
24/11/12 00:03:06 WARN DAGScheduler: Broadcasting large task binary with size 1357.8 KiB
24/11/12 00:03:12 WARN DAGScheduler: Broadcasting large task binary with size 1359.0 KiB
24/11/12 00:03:14 WARN DAGScheduler: Broadcasting large task binary with size 1360.5 KiB
24/11/12 00:03:16 WARN DAGScheduler: Broadcasting large task binary with size 1363.4 KiB
24/11/12 00:03:18 WARN DAGScheduler: Broadcasting large task binary with size 1369.3 KiB
24/11/12 00:03:21 WARN DAGScheduler: Broadcasting large task binary with size 1313.3 KiB
24/11/12 00:03:22 WARN DAGScheduler: Broadcasting large task binary with size 1334.3 KiB
24/11/12 00:03:23 WAR

24/11/12 00:05:06 WARN DAGScheduler: Broadcasting large task binary with size 1360.6 KiB
24/11/12 00:05:07 WARN DAGScheduler: Broadcasting large task binary with size 1364.3 KiB
24/11/12 00:05:09 WARN DAGScheduler: Broadcasting large task binary with size 1371.7 KiB
24/11/12 00:05:10 WARN DAGScheduler: Broadcasting large task binary with size 1386.4 KiB
24/11/12 00:05:12 WARN DAGScheduler: Broadcasting large task binary with size 1415.8 KiB
24/11/12 00:05:13 WARN DAGScheduler: Broadcasting large task binary with size 1474.5 KiB
24/11/12 00:05:15 WARN DAGScheduler: Broadcasting large task binary with size 1593.6 KiB
24/11/12 00:05:18 WARN DAGScheduler: Broadcasting large task binary with size 1819.3 KiB
24/11/12 00:05:23 WARN DAGScheduler: Broadcasting large task binary with size 1334.3 KiB
24/11/12 00:05:24 WARN DAGScheduler: Broadcasting large task binary with size 1335.4 KiB
24/11/12 00:05:25 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
24/11/12 00:05:25 WAR

24/11/12 00:07:38 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
24/11/12 00:07:38 WARN DAGScheduler: Broadcasting large task binary with size 1342.0 KiB
24/11/12 00:07:39 WARN DAGScheduler: Broadcasting large task binary with size 1347.0 KiB
24/11/12 00:07:40 WARN DAGScheduler: Broadcasting large task binary with size 1357.8 KiB
24/11/12 00:07:42 WARN DAGScheduler: Broadcasting large task binary with size 1359.0 KiB
24/11/12 00:07:43 WARN DAGScheduler: Broadcasting large task binary with size 1360.5 KiB
24/11/12 00:07:43 WARN DAGScheduler: Broadcasting large task binary with size 1363.4 KiB
24/11/12 00:07:44 WARN DAGScheduler: Broadcasting large task binary with size 1369.3 KiB
24/11/12 00:07:45 WARN DAGScheduler: Broadcasting large task binary with size 1381.1 KiB
24/11/12 00:07:47 WARN DAGScheduler: Broadcasting large task binary with size 1404.7 KiB
24/11/12 00:07:48 WARN DAGScheduler: Broadcasting large task binary with size 1452.3 KiB
24/11/12 00:07:49 WAR

24/11/12 00:09:33 WARN DAGScheduler: Broadcasting large task binary with size 1371.8 KiB
24/11/12 00:09:34 WARN DAGScheduler: Broadcasting large task binary with size 1386.5 KiB
24/11/12 00:09:35 WARN DAGScheduler: Broadcasting large task binary with size 1415.9 KiB
24/11/12 00:09:37 WARN DAGScheduler: Broadcasting large task binary with size 1474.6 KiB
24/11/12 00:09:39 WARN DAGScheduler: Broadcasting large task binary with size 1334.3 KiB
24/11/12 00:09:40 WARN DAGScheduler: Broadcasting large task binary with size 1335.4 KiB
24/11/12 00:09:41 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
24/11/12 00:09:41 WARN DAGScheduler: Broadcasting large task binary with size 1342.0 KiB
24/11/12 00:09:42 WARN DAGScheduler: Broadcasting large task binary with size 1347.0 KiB
24/11/12 00:09:43 WARN DAGScheduler: Broadcasting large task binary with size 1359.0 KiB
24/11/12 00:09:45 WARN DAGScheduler: Broadcasting large task binary with size 1363.4 KiB
24/11/12 00:09:46 WAR

24/11/12 00:11:34 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
24/11/12 00:11:34 WARN DAGScheduler: Broadcasting large task binary with size 1342.0 KiB
24/11/12 00:11:34 WARN DAGScheduler: Broadcasting large task binary with size 1347.0 KiB
24/11/12 00:11:36 WARN DAGScheduler: Broadcasting large task binary with size 1357.8 KiB
24/11/12 00:11:38 WARN DAGScheduler: Broadcasting large task binary with size 1359.0 KiB
24/11/12 00:11:38 WARN DAGScheduler: Broadcasting large task binary with size 1360.5 KiB
24/11/12 00:11:39 WARN DAGScheduler: Broadcasting large task binary with size 1363.5 KiB
24/11/12 00:11:40 WARN DAGScheduler: Broadcasting large task binary with size 1369.4 KiB
24/11/12 00:11:42 WARN DAGScheduler: Broadcasting large task binary with size 1381.1 KiB
24/11/12 00:11:43 WARN DAGScheduler: Broadcasting large task binary with size 1404.7 KiB
24/11/12 00:11:46 WARN DAGScheduler: Broadcasting large task binary with size 1334.3 KiB
24/11/12 00:11:47 WAR

24/11/12 00:13:32 WARN DAGScheduler: Broadcasting large task binary with size 1358.2 KiB
24/11/12 00:13:34 WARN DAGScheduler: Broadcasting large task binary with size 1360.6 KiB
24/11/12 00:13:35 WARN DAGScheduler: Broadcasting large task binary with size 1364.3 KiB
24/11/12 00:13:36 WARN DAGScheduler: Broadcasting large task binary with size 1371.7 KiB
24/11/12 00:13:37 WARN DAGScheduler: Broadcasting large task binary with size 1386.4 KiB
24/11/12 00:13:39 WARN DAGScheduler: Broadcasting large task binary with size 1334.3 KiB
24/11/12 00:13:40 WARN DAGScheduler: Broadcasting large task binary with size 1335.4 KiB
24/11/12 00:13:41 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
24/11/12 00:13:41 WARN DAGScheduler: Broadcasting large task binary with size 1342.0 KiB
24/11/12 00:13:41 WARN DAGScheduler: Broadcasting large task binary with size 1347.0 KiB
24/11/12 00:13:43 WARN DAGScheduler: Broadcasting large task binary with size 1358.9 KiB
24/11/12 00:13:45 WAR

24/11/12 00:15:25 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
24/11/12 00:15:31 WARN DAGScheduler: Broadcasting large task binary with size 1334.3 KiB
24/11/12 00:15:31 WARN DAGScheduler: Broadcasting large task binary with size 1335.4 KiB
24/11/12 00:15:33 WARN DAGScheduler: Broadcasting large task binary with size 1317.9 KiB
24/11/12 00:15:33 WARN DAGScheduler: Broadcasting large task binary with size 1318.0 KiB
24/11/12 00:15:34 WARN DAGScheduler: Broadcasting large task binary with size 1323.0 KiB
24/11/12 00:15:35 WARN DAGScheduler: Broadcasting large task binary with size 1334.9 KiB
24/11/12 00:15:37 WARN DAGScheduler: Broadcasting large task binary with size 1339.3 KiB
24/11/12 00:15:38 WARN DAGScheduler: Broadcasting large task binary with size 1346.7 KiB
24/11/12 00:15:39 WARN DAGScheduler: Broadcasting large task binary with size 1361.5 KiB
24/11/12 00:15:40 WARN DAGScheduler: Broadcasting large task binary with size 1390.9 KiB
24/11/12 00:15:42 WARN D

Random Forest Model: Mean Squared Error for Test dataset= 0.11205458857127683


### Pytorch ML models

**convert spark dataset into pandas for using pytorch**

In [30]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 1.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━ 621.1/664.8 MB 45.9 MB/s eta 0:00:01

24/11/12 00:40:12 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_7507_12 !


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 664.8/664.8 MB 66.1 MB/s eta 0:00:0100:01

24/11/12 00:40:14 WARN YarnAllocator: Container from a bad node: container_1731358515123_0001_01_000008 on host: cluster-f87f-m.us-central1-f.c.forward-byte-434113-d3.internal. Exit status: 143. Diagnostics: [2024-11-12 00:40:13.634]Container killed on request. Exit code is 143
[2024-11-12 00:40:13.635]Container exited with a non-zero exit code 143. 
[2024-11-12 00:40:13.637]Killed by external signal
.
24/11/12 00:40:14 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 8 for reason Container from a bad node: container_1731358515123_0001_01_000008 on host: cluster-f87f-m.us-central1-f.c.forward-byte-434113-d3.internal. Exit status: 143. Diagnostics: [2024-11-12 00:40:13.634]Container killed on request. Exit code is 143
[2024-11-12 00:40:13.635]Container exited with a non-zero exit code 143. 
[2024-11-12 00:40:13.637]Killed by external signal
.
24/11/12 00:40:14 ERROR YarnScheduler: Lost executor 8 on cluster-f87f-m.us-central1-f.c.forward-byte-434113-

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 95.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 3.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12.1
    Uninstalling sympy-1.12.1:
      

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

train = train_df.select("features", "outcome").distinct().toPandas()
test = test_df.select("features", "outcome").distinct().toPandas()

# expand the features column into separate 57 columns
features_expanded = pd.DataFrame(train['features'].tolist(), index=train.index)
train = pd.concat([features_expanded, train['outcome']], axis=1)
features_expanded_test = pd.DataFrame(test['features'].tolist(), index=test.index)
test = pd.concat([features_expanded_test, test['outcome']], axis=1)

# Define X_train, y_train, X_test, y_test
X_train = train.drop(columns=['outcome'])
y_train = train['outcome']
X_test = test.drop(columns=['outcome'])
y_test = test['outcome']

print(X_train.shape) 
print(y_train.shape)

24/11/12 00:42:33 WARN DAGScheduler: Broadcasting large task binary with size 1308.0 KiB
24/11/12 00:42:40 WARN DAGScheduler: Broadcasting large task binary with size 1314.1 KiB
24/11/12 00:42:52 WARN DAGScheduler: Broadcasting large task binary with size 1299.1 KiB
24/11/12 00:43:24 WARN DAGScheduler: Broadcasting large task binary with size 1063.4 KiB


(37648, 57)
(37648,)


**Linear Regression model in Pytorch**

In [32]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.linear(x)

In [34]:
input_dim = X_train.shape[1]
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)

#### LR Experiment1 ####
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1)
num_epochs = 1000

for epoch in range(num_epochs):
    model.train() 
    
    outputs = model(torch.tensor(X_train.values, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), torch.tensor(y_train.values, dtype=torch.float32))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 3.4282
Epoch [200/1000], Loss: 0.5718
Epoch [300/1000], Loss: 0.5065
Epoch [400/1000], Loss: 0.4627
Epoch [500/1000], Loss: 0.4329
Epoch [600/1000], Loss: 0.4122
Epoch [700/1000], Loss: 0.3976
Epoch [800/1000], Loss: 0.3868
Epoch [900/1000], Loss: 0.3784
Epoch [1000/1000], Loss: 0.3718


In [35]:
#### LR Experiment2 ####
LR_model2 = LinearRegressionModel(input_dim, output_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(LR_model2.parameters(), lr=0.5)
num_epochs = 2000

for epoch in range(num_epochs):
    LR_model2.train() 
    
    outputs = LR_model2(torch.tensor(X_train.values, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), torch.tensor(y_train.values, dtype=torch.float32))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [200/2000], Loss: 0.5284
Epoch [400/2000], Loss: 0.4485
Epoch [600/2000], Loss: 0.4073
Epoch [800/2000], Loss: 0.3838
Epoch [1000/2000], Loss: 0.3693
Epoch [1200/2000], Loss: 0.3597
Epoch [1400/2000], Loss: 0.3533
Epoch [1600/2000], Loss: 0.3490
Epoch [1800/2000], Loss: 0.3460
Epoch [2000/2000], Loss: 482.6470


In [36]:
## use LR experiment1 as the best model
model.eval()
with torch.no_grad():
    y_pred = model(torch.tensor(X_test.values, dtype=torch.float32))
    loss = criterion(y_pred.squeeze(), torch.tensor(y_test.values, dtype=torch.float32))
    print(f"The test loss is: {loss.item():.4f}")

The test loss is: 0.3745


### MLP models with different tuning hypermeters in Pytorch

In [37]:
import torch
import torch.nn as nn

class MLPRegressor(nn.Module):
    def __init__(self, input_dim):
        super(MLPRegressor, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 128)
        self.hidden2 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        return self.output(x)

input_dim = X_train.shape[1]
model = MLPRegressor(input_dim)

##### MLP Experiment1 ####
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    outputs = model(torch.tensor(X_train.values, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), torch.tensor(y_train.values, dtype=torch.float32))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.9944
Epoch [200/1000], Loss: 0.9943
Epoch [300/1000], Loss: 0.9943
Epoch [400/1000], Loss: 0.9943
Epoch [500/1000], Loss: 0.9943
Epoch [600/1000], Loss: 0.9943
Epoch [700/1000], Loss: 0.9943
Epoch [800/1000], Loss: 0.9943
Epoch [900/1000], Loss: 0.9943
Epoch [1000/1000], Loss: 0.9943


In [38]:
##### MLP Experiment2 ####
MLP_model2 = MLPRegressor(input_dim)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(MLP_model2.parameters(), lr=0.01)

num_epochs = 2000
for epoch in range(num_epochs):
    MLP_model2.train()
    outputs = MLP_model2(torch.tensor(X_train.values, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), torch.tensor(y_train.values, dtype=torch.float32))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/2000], Loss: 0.7187
Epoch [200/2000], Loss: 0.3877
Epoch [300/2000], Loss: 0.3023
Epoch [400/2000], Loss: 0.2645
Epoch [500/2000], Loss: 0.2537
Epoch [600/2000], Loss: 0.2364
Epoch [700/2000], Loss: 0.2223
Epoch [800/2000], Loss: 0.2273
Epoch [900/2000], Loss: 0.2130
Epoch [1000/2000], Loss: 0.2156
Epoch [1100/2000], Loss: 0.2052
Epoch [1200/2000], Loss: 0.2024
Epoch [1300/2000], Loss: 0.1998
Epoch [1400/2000], Loss: 0.2111
Epoch [1500/2000], Loss: 0.1971
Epoch [1600/2000], Loss: 0.2034
Epoch [1700/2000], Loss: 0.1913
Epoch [1800/2000], Loss: 0.1900
Epoch [1900/2000], Loss: 0.2210
Epoch [2000/2000], Loss: 0.1866


In [39]:
## use MLP experiment2 as the best model
MLP_model2.eval()
with torch.no_grad():
    test_outputs = MLP_model2(torch.tensor(X_test.values, dtype=torch.float32))
    test_loss = criterion(test_outputs.squeeze(), torch.tensor(y_test.values, dtype=torch.float32))
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.1938


## Task IV

Deploy 3 tasks to the Cloud

Training models on cloud gives different loss values to local training for each models. However, the random forest model with SparkML still be the best model with the different ranking; MLP with Pytorch, linear regression with SparkML, and Linear regrssion with Pytorch respectively.